In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

# Credenciales (ajusta)
USER = "root"
PASSWORD = "05AcdT8*"
HOST = "127.0.0.1"   # mejor que 'localhost'
DB = "worldbank"

engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{DB}")

# Pequeña ayuda para leer SQL con params y devolver DataFrame
def q(sql, params=None):
    return pd.read_sql(sql, engine, params=params)


Matplotlib is building the font cache; this may take a moment.


In [4]:
def show_columns(table):
    try:
        return pd.read_sql(f"SHOW COLUMNS FROM {table};", engine)
    except Exception as e:
        return pd.DataFrame({"error":[str(e)]})

print("observations")
display(show_columns("observations"))

print("\nindicators")
display(show_columns("indicators"))

print("\ncountries")
display(show_columns("countries"))


observations


,Field,Type,Null,Key,Default,Extra
0,country_code,char(3),YES,MUL,None,
1,indicator_code,varchar(50),YES,MUL,None,
2,year,int,YES,,None,
3,value,"decimal(20,2)",YES,,None,



indicators


,Field,Type,Null,Key,Default,Extra
0,indicator_code,varchar(50),NO,PRI,None,
1,indicator_name,varchar(255),YES,,None,
2,unit,varchar(255),YES,,None,
3,topic,varchar(100),YES,,None,



countries


,Field,Type,Null,Key,Default,Extra
0,country_code,char(3),NO,PRI,None,
1,country_name,varchar(100),YES,,None,


In [5]:
# Construye SELECTs "tolerantes" a nombres alternativos de columnas

# OBSERVATIONS -> country_name, indicator_code, year, value
OBS_SELECT = """
SELECT
  /* country_name */
  COALESCE(o.country_name, o.country, c.name, o.country_iso3)      AS country_name,
  /* indicator_code */
  COALESCE(o.indicator_code, o.indicator)                           AS indicator_code,
  /* year */
  o.year                                                            AS year,
  /* value */
  o.value                                                           AS value
FROM observations o
LEFT JOIN countries c
  ON c.iso3 = o.country_iso3
"""

# INDICATORS -> indicator_code, indicator_name, unit
IND_SELECT = """
SELECT
  COALESCE(i.indicator_code, i.indicator)               AS indicator_code,
  COALESCE(i.indicator_name, i.name)                    AS indicator_name,
  COALESCE(i.unit, '')                                  AS unit
FROM indicators i
"""

# PAÍSES (opcional, por si querés lista bonita)
COUNTRY_SELECT = """
SELECT DISTINCT
  COALESCE(c.name, o.country_name, o.country, o.country_iso3) AS country_name
FROM observations o
LEFT JOIN countries c
  ON c.iso3 = o.country_iso3
WHERE COALESCE(c.name, o.country_name, o.country, o.country_iso3) IS NOT NULL
ORDER BY 1
"""


In [10]:
# === CELDA 4: Construcción segura de catálogos ===

# Función auxiliar para elegir columnas existentes
def pick_column(colnames, candidates):
    for c in candidates:
        if c in colnames:
            return c
    return None

# Obtener columnas reales de cada tabla
obs_cols = [c["Field"] for c in q("SHOW COLUMNS FROM observations;").to_dict("records")]
ind_cols = [c["Field"] for c in q("SHOW COLUMNS FROM indicators;").to_dict("records")]
try:
    ctr_cols = [c["Field"] for c in q("SHOW COLUMNS FROM countries;").to_dict("records")]
except:
    ctr_cols = []

# Identificar columnas clave
o_country_iso3 = pick_column(obs_cols, ["country_iso3","country_code"])
o_country_name = pick_column(obs_cols, ["country_name","country"])
o_indicator_code = pick_column(obs_cols, ["indicator_code","indicator"])
o_year   = pick_column(obs_cols, ["year","anio"])
o_value  = pick_column(obs_cols, ["value","valor"])

i_code = pick_column(ind_cols, ["indicator_code","indicator","code","id"])
i_name = pick_column(ind_cols, ["indicator_name","name","nombre"])
i_unit = pick_column(ind_cols, ["unit","unidad"])

c_iso3 = pick_column(ctr_cols, ["iso3","country_code"])
c_name = pick_column(ctr_cols, ["name","country_name","pais"])

# === Lista de países ===
if c_iso3 and c_name and o_country_iso3:
    sql = f"""
        SELECT DISTINCT c.{c_name} AS name
        FROM countries c
        JOIN observations o ON c.{c_iso3} = o.{o_country_iso3}
        ORDER BY 1;
    """
    countries = q(sql)["name"].tolist()
elif o_country_name:
    sql = f"""
        SELECT DISTINCT o.{o_country_name} AS name
        FROM observations o
        ORDER BY 1;
    """
    countries = q(sql)["name"].tolist()
elif o_country_iso3:
    sql = f"""
        SELECT DISTINCT o.{o_country_iso3} AS name
        FROM observations o
        ORDER BY 1;
    """
    countries = q(sql)["name"].tolist()
else:
    countries = []

print("Países detectados:", countries)

# === Indicadores de economía ===
codes = ("NY.GDP.MKTP.CD","NY.GDP.PCAP.CD","NE.EXP.GNFS.ZS","NY.GDP.DEFL.KD.ZG")
if i_code:
    sql = f"""
        SELECT {i_code} AS indicator_code,
               COALESCE({i_name},'') AS indicator_name,
               COALESCE({i_unit},'') AS unit
        FROM indicators
        WHERE {i_code} IN {codes};
    """
    inds = q(sql)
    inds["label"] = inds["indicator_code"] + " – " + inds["indicator_name"]
else:
    inds = pd.DataFrame(columns=["indicator_code","indicator_name","unit","label"])

display(inds)

# === Rango de años ===
if o_year:
    sql = f"SELECT MIN({o_year}) AS y1, MAX({o_year}) AS y2 FROM observations;"
    years = q(sql)
    year_min, year_max = int(years.iloc[0]["y1"]), int(years.iloc[0]["y2"])
else:
    year_min, year_max = 2000, 2024

print("Años:", year_min, "→", year_max)


Países detectados: ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Paraguay', 'Uruguay']


,indicator_code,indicator_name,unit,label
0,NE.EXP.GNFS.ZS,Exportaciones de bienes y servicios (% del PIB),Porcentaje del PIB (%),NE.EXP.GNFS.ZS – Exportaciones de bienes y ser...
1,NY.GDP.DEFL.KD.ZG,"Inflación, deflactor del PIB (% anual)",Porcentaje (%),"NY.GDP.DEFL.KD.ZG – Inflación, deflactor del P..."
2,NY.GDP.MKTP.CD,PIB (dólares estadounidenses a precios actuales),Dólares estadounidenses corrientes (US$),NY.GDP.MKTP.CD – PIB (dólares estadounidenses ...
3,NY.GDP.PCAP.CD,PIB per cápita (dólares estadounidenses a prec...,Dólares estadounidenses corrientes por habitan...,NY.GDP.PCAP.CD – PIB per cápita (dólares estad...


Años: 2000 → 2024


In [ ]:
# === CELDA 5: Dashboard con widgets  ===
import ipywidgets as w
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd

# ---- Validaciones mínimas del esquema detectado en la Celda 4 ----
required_obs = [o_year, o_value, o_indicator_code]
required_ind = [i_code]
if any(x is None for x in required_obs) or any(x is None for x in required_ind):
    raise SystemExit("Faltan columnas clave en el esquema. Repetí la Celda 4 y revisá que existan "
                     "en 'observations' (year/value/indicator_code) y en 'indicators' (indicator_code).")

# ---- Función para cargar la serie temporal según el esquema real ----
def load_timeseries(ind_code, countries_sel, y1, y2):
    if not countries_sel:
        return pd.DataFrame()

    # Elegir cómo obtener el nombre del país (join a countries si es posible)
    if c_iso3 and c_name and o_country_iso3:
        country_expr = f"c.{c_name}"
        join_c = f"JOIN countries c ON c.{c_iso3} = o.{o_country_iso3}"
    elif o_country_name:  # si observations ya trae el nombre
        country_expr = f"o.{o_country_name}"
        join_c = ""
    elif o_country_iso3:  # último recurso: usar el ISO3 como "nombre"
        country_expr = f"o.{o_country_iso3}"
        join_c = ""
    else:
        # No hay forma de construir el país
        return pd.DataFrame()

    name_expr = f"i.{i_name}" if i_name else "NULL"
    unit_expr = f"i.{i_unit}" if i_unit else "NULL"

    sql = f"""
        SELECT
            {country_expr} AS country_name,
            o.{o_year}      AS year,
            o.{o_value}     AS value,
            i.{i_code}      AS indicator_code,
            {name_expr}     AS indicator_name,
            COALESCE({unit_expr}, '') AS unit
        FROM observations o
        {join_c}
        JOIN indicators i ON i.{i_code} = o.{o_indicator_code}
        WHERE i.{i_code} = :code
          AND {country_expr} IN :countries
          AND o.{o_year} BETWEEN :y1 AND :y2
        ORDER BY 1, 2;
    """
    df = q(sql, {"code": ind_code, "countries": tuple(countries_sel), "y1": int(y1), "y2": int(y2)})
    if df.empty:
        return df

    # Tipos y limpieza
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna(subset=["year", "value"])
    df["year"] = df["year"].astype(int)
    return df

# ---- Preparar widgets usando los catálogos construidos en la Celda 4 ----
if inds.empty:
    raise SystemExit("No hay indicadores disponibles (inds está vacío). Verificá la Celda 4.")
if not countries:
    raise SystemExit("No hay países detectados. Verificá la Celda 4 y que observations tenga datos.")

w_countries = w.SelectMultiple(
    options=countries,
    value=tuple(countries[: min(3, len(countries))]),
    description="Países",
    rows=7,
    layout=w.Layout(width="40%"),
)

w_indicator = w.Dropdown(
    options=inds["label"].tolist(),
    value=inds["label"].iloc[0],
    description="Indicador",
    layout=w.Layout(width="90%"),
)

w_rango = w.IntRangeSlider(
    value=[max(year_min, year_max - 10), year_max],
    min=year_min,
    max=year_max,
    step=1,
    description="Años",
    continuous_update=False,
    layout=w.Layout(width="70%"),
)

ui = w.VBox([w_indicator, w_countries, w_rango])
out = w.Output()

def render(_=None):
    with out:
        clear_output(wait=True)

        sel_countries = list(w_countries.value)
        if not sel_countries:
            print("Elegí al menos un país.")
            return

        sel_indicator_code = w_indicator.value.split(" – ", 1)[0]
        y1, y2 = w_rango.value

        df = load_timeseries(sel_indicator_code, sel_countries, y1, y2)
        if df.empty:
            print("No hay datos para el filtro elegido.")
            return

        unit = ""
        if "unit" in df.columns and df["unit"].notna().any():
            unit = df["unit"].dropna().iloc[0]

        title = f"{sel_indicator_code} – {unit}" if unit else sel_indicator_code

        # --- Serie temporal por país ---
        plt.figure(figsize=(9, 4))
        for country, dsub in df.groupby("country_name"):
            dsub = dsub.sort_values("year")
            plt.plot(dsub["year"], dsub["value"], marker="o", label=country)
        plt.title(title)
        plt.xlabel("Año")
        plt.ylabel(f"Valor {f'({unit})' if unit else ''}")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

        # --- Ranking del último año del rango ---
        last_year = int(df["year"].max())
        rank = df[df["year"] == last_year].sort_values("value", ascending=False)

        plt.figure(figsize=(8, 4))
        plt.barh(rank["country_name"], rank["value"])
        plt.gca().invert_yaxis()
        plt.title(f"Ranking {last_year} – {title}")
        plt.xlabel(f"Valor {f'({unit})' if unit else ''}")
        plt.show()

        # --- Tabla de datos ordenada ---
        display(df.sort_values(["country_name", "year"]).reset_index(drop=True))

# Conectar eventos
for widget in (w_countries, w_indicator, w_rango):
    widget.observe(render, names="value")

display(ui, out)
render()


Output()

In [15]:
import ipywidgets as w
print("ipywidgets cargado OK:", w.__version__)

ipywidgets cargado OK: 8.1.7


In [ ]:
# === CELDA 4: Construcción segura de catálogos ===

# Función auxiliar para elegir columnas existentes
def pick_column(colnames, candidates):
    for c in candidates:
        if c in colnames:
            return c
    return None

# Obtener columnas reales de cada tabla
obs_cols = [c["Field"] for c in q("SHOW COLUMNS FROM observations;").to_dict("records")]
ind_cols = [c["Field"] for c in q("SHOW COLUMNS FROM indicators;").to_dict("records")]
try:
    ctr_cols = [c["Field"] for c in q("SHOW COLUMNS FROM countries;").to_dict("records")]
except:
    ctr_cols = []

# Identificar columnas clave
o_country_iso3 = pick_column(obs_cols, ["country_iso3","country_code"])
o_country_name = pick_column(obs_cols, ["country_name","country"])
o_indicator_code = pick_column(obs_cols, ["indicator_code","indicator"])
o_year   = pick_column(obs_cols, ["year","anio"])
o_value  = pick_column(obs_cols, ["value","valor"])

i_code = pick_column(ind_cols, ["indicator_code","indicator","code","id"])
i_name = pick_column(ind_cols, ["indicator_name","name","nombre"])
i_unit = pick_column(ind_cols, ["unit","unidad"])

c_iso3 = pick_column(ctr_cols, ["iso3","country_code"])
c_name = pick_column(ctr_cols, ["name","country_name","pais"])

# === Lista de países ===
if c_iso3 and c_name and o_country_iso3:
    countries = q(f"SELECT DISTINCT c.{c_name} FROM countries c "
                  f"JOIN observations o ON c.{c_iso3}=o.{o_country_iso3} "
                  f"ORDER BY 1;")[c_name].tolist()
elif o_country_name:
    countries = q(f"SELECT DISTINCT {o_country_name} AS name "
                  f"FROM observations ORDER BY 1;")["name"].tolist()
elif o_country_iso3:
    countries = q(f"SELECT DISTINCT {o_country_iso3} AS name "
                  f"FROM observations


SyntaxError: unterminated f-string literal (detected at line 42) (1955571569.py, line 42)